In [4]:
import os
os.chdir("../src")

import tensorflow as tf
import numpy as np
from dataloader import qm9_parse, qm9_fetch
import dmol


In [5]:
qm9_records = qm9_fetch()

data = qm9_parse(qm9_records)


Found existing record file, delete if you want to re-fetch


In [11]:
# for d in data:
#     print(d)
#     break


In [6]:
#Train with 5000 and use 1000 for test
shuffled_data = data.shuffle(700, reshuffle_each_iteration=False)
test_set = shuffled_data.take(100)
valid_set = shuffled_data.skip(100).take(100)
train_set = shuffled_data.skip(200).take(500)


In [7]:
import torch
import torch.nn as nn

# Define the embedding layer outside the loop
embedding_layer = nn.Embedding(16, 128)

def convert_record(d):
    # break up record
    (e, x), y = d
    e = e.numpy()
    x= x.numpy()
    r= x[:,:3]
    # embedding= nn.Embedding(100,128,padding_idx=0)
    e=torch.tensor(e)
    e = embedding(e)
    r =torch.tensor(r)
    return(e,r),y.numpy()[13]

# for d in data:
#     print(d)
#     (e, x), y = convert_record(d)
#     print("Embedded Atomic Numbers:\n", e)
#     print("Coordinates:\n", x)
#     print("Label:", y)
#     print()
#     break


In [8]:
def convert_record(d):
    # break up record
    (e, x), y = d
    e = e.numpy()
    x= x.numpy()
    r= x[:,:3]
    embedding= nn.Embedding(100,128,padding_idx=0)
    e=torch.tensor(e)
    s_j = embedding(e)
    r =torch.tensor(r)
    return(s_j,r),y.numpy()[13]


In [9]:
def x2e(x):
    """convert xyz coordinates to inverse pairwise distance"""
    r2 = torch.sum((x - x[:, None, :]) ** 2, axis=-1)
    # r2 = jnp.sum((x - x[:, jnp.newaxis, :]) ** 2, axis=-1)
    # e = jnp.where(r2 != 0, 1 / r2, 0.0)
    e = torch.sqrt(torch.where(r2 != 0, r2, 0.0))

    return e


In [10]:

for d in data:
    (s_j,r),response_var = convert_record(d)
    break


In [42]:
print(r[0])
print(len(r))


tensor([-0.0127,  1.0858,  0.0080])
5


In [12]:
for atom in r:
    print(atom)


tensor([-0.0127,  1.0858,  0.0080])
tensor([ 0.0022, -0.0060,  0.0020])
tensor([1.0117e+00, 1.4638e+00, 2.7657e-04])
tensor([-0.5408,  1.4475, -0.8766])
tensor([-0.5238,  1.4379,  0.9064])


In [134]:
def create_r_ij(r, r_cut=5.0):
    r_ij = torch.tensor([])
    for i in range(len(r)):
        for j in range(len(r)):
            if torch.norm(r[i] - r[j]) <r_cut:
                r_ij = torch.cat((r_ij, r[i] - r[j])) #important
            else:
                continue
        return r_ij


In [159]:
# vi bruger denne
def create_r_ij(r, r_cut=5.0):
    r_ij = []
    for i in range(len(r)):
        for j in range(len(r)):
            if torch.norm(r[i] - r[j]) <r_cut:
                r_ij.append(r[i] - r[j]) #important
            else:
                continue
        return r_ij


In [160]:
create_r_ij(r,r_cut=5.0)


[tensor([0., 0., 0.]),
 tensor([-0.0148,  1.0918,  0.0060]),
 tensor([-1.0244, -0.3779,  0.0077]),
 tensor([ 0.5281, -0.3617,  0.8846]),
 tensor([ 0.5111, -0.3521, -0.8984])]

In [176]:
# define input

r_ij = create_r_ij(r,r_cut=5.0) #vectorization
A = x2e(r) # adjacency matrix
r_pos = r # position matrix
s_j = s_j
r_cut = 5.0


In [187]:

r_ij_tensor = torch.stack(r_ij)
r_ij_tensor


tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0148,  1.0918,  0.0060],
        [-1.0244, -0.3779,  0.0077],
        [ 0.5281, -0.3617,  0.8846],
        [ 0.5111, -0.3521, -0.8984]])

In [191]:
v_norm = r_ij_tensor/(torch.norm(r_ij_tensor))
v_norm


tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0068,  0.4999,  0.0028],
        [-0.4691, -0.1731,  0.0035],
        [ 0.2418, -0.1656,  0.4051],
        [ 0.2340, -0.1612, -0.4114]])

In [192]:

#Message block

v_norm = r_ij_tensor/(torch.norm(r_ij_tensor))

v_j = torch.zeros(128)

class phi(nn.Module):

     def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU

        self.net = nn.Sequential(
            nn.Linear(s_j, 128),
            activation_fn(),
            nn.Linear(128, 384),
        )
class w(nn.Module):
    def __init__(self):
        super().__init__()
        RBF = []
        for n in range(21):
            if i>0:
                RBF.append = torch.sin((n*torch.pi()/r_cut)*r_ij)/r_ij
            else:
                continue
        f_c=0.5*torch.cos(torch.pi()*r_ij/r_cut)+1
        fcRBF = RBF * f_c
        self.fcRBF=nn.linear(fcRBF,384)


def forward(self, input1, input2):
    phi = self.net(input1)
    w = self.net(input2)
    output = phi * w
    output = torch.split(output,3)
    output1 = output[0] * v_j
    s_m = torch.sum(output[1]) + s_j
    output3 = output[2]*v_norm
    v_m= torch.sum(output1+output3) + v_j
    return v_m, s_m


In [193]:
v_m, s_m = forward(self, s_j, r_ij) # input1, input2


NameError: name 'self' is not defined

In [ ]:

#Update block

class u(nn.Module):
      def __init__(self):
        super().__init__()
        self.net = nn.Linear(v_m,128)
class v(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Linear(v_m,128)
class s(nn.Module):
    def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU
        self.net = nn.Sequential(
            torch.stack(v_norm,s_m),
            nn.Linear(256,128),
            activation_fn(),
            nn.Linear(128,384))

def forward(self,input3,input4,input5):
    U = self.net(input3)
    V = self.net(input4)
    S = self.net(input5)
    V_dup = V.repeat(128,2)
    S = torch.split(S,3)
    output4 = S[0]*U
    output5 = S[1]*V_dup
    output6 = S[2]+output5
    v_u = output4 + v_m
    s_u = output6 + s_m
    return v_u, s_u


In [ ]:

#Class pain

#Kør 3x message og 3x update (skiftesvis)

#Linear, SiLU, Linear, output




# class PAINN(nn.Module):
#     def __init__(self, n_basis=int(128)):
#         super(PAINN, self).__init__()
#         self.n_basis = n_basis
#         self.embedding = nn.Embedding(16,n_basis,padding_idx=0)

#     def x2e(r):
#     """convert xyz coordinates to pairwise distance"""
#     r2 = np.sqrt(np.sum((r - r[:, np.newaxis, :]) ** 2, axis=-1))
#     r_cut = 5
#     r_ij = np.where(r2 < r_cut)
#     return r_ij



#     def init_weights(g, n, m):
#     we = np.random.normal(size=(n, m), scale=1e-1)
#     wb = np.random.normal(size=(m), scale=1e-1)
#     wv = np.random.normal(size=(m, n), scale=1e-1)
#     wu = np.random.normal(size=(n, g), scale=1e-1)
#     return [we, wb, wv, wu]


In [ ]:
class update(nn.Module):

class message(nn.Module):

     def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU

        self.net = nn.Sequential(
            nn.Linear(s_j, 128),
            activation_fn(),
            nn.Linear(128, 128),
        )



class PAINN(nn.Module):
    def __init__(self, n_basis=int(128)):
        super(PAINN, self).__init__()
        self.n_basis = n_basis
        self.embedding = nn.Embedding(16,n_basis,padding_idx=0)

    def x2e(r):
    """convert xyz coordinates to pairwise distance"""
    r2 = np.sqrt(np.sum((r - r[:, np.newaxis, :]) ** 2, axis=-1))
    r_cut = 5
    r_ij = np.where(r2 < r_cut)
    return r_ij

    def init_weights(g, n, m):
    we = np.random.normal(size=(n, m), scale=1e-1)
    wb = np.random.normal(size=(m), scale=1e-1)
    wv = np.random.normal(size=(m, n), scale=1e-1)
    wu = np.random.normal(size=(n, g), scale=1e-1)
    return [we, wb, wv, wu]


In [12]:
from typing import Callable, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

import schnetpack.properties as properties
import schnetpack.nn as snn

__all__ = ["PaiNN", "PaiNNInteraction", "PaiNNMixing"]


class PaiNNInteraction(nn.Module):
    r"""PaiNN interaction block for modeling equivariant interactions of atomistic systems."""

    def __init__(self, n_atom_basis: int, activation: Callable):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
            activation: if None, no activation function is used.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNNInteraction, self).__init__()
        self.n_atom_basis = n_atom_basis

        self.interatomic_context_net = nn.Sequential(
            snn.Dense(n_atom_basis, n_atom_basis, activation=activation),
            snn.Dense(n_atom_basis, 3 * n_atom_basis, activation=None),
        )

    def forward(
        self,
        q: torch.Tensor,
        mu: torch.Tensor,
        Wij: torch.Tensor,
        dir_ij: torch.Tensor,
        idx_i: torch.Tensor,
        idx_j: torch.Tensor,
        n_atoms: int,
    ):
        """Compute interaction output.

        Args:
            q: scalar input values
            mu: vector input values
            Wij: filter
            idx_i: index of center atom i
            idx_j: index of neighbors j

        Returns:
            atom features after interaction
        """
        # inter-atomic
        x = self.interatomic_context_net(q)
        xj = x[idx_j]
        muj = mu[idx_j]
        x = Wij * xj

        dq, dmuR, dmumu = torch.split(x, self.n_atom_basis, dim=-1)
        dq = snn.scatter_add(dq, idx_i, dim_size=n_atoms)
        dmu = dmuR * dir_ij[..., None] + dmumu * muj
        dmu = snn.scatter_add(dmu, idx_i, dim_size=n_atoms)

        q = q + dq
        mu = mu + dmu

        return q, mu


class PaiNNMixing(nn.Module):
    r"""PaiNN interaction block for mixing on atom features."""

    def __init__(self, n_atom_basis: int, activation: Callable, epsilon: float = 1e-8):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
            activation: if None, no activation function is used.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNNMixing, self).__init__()
        self.n_atom_basis = n_atom_basis

        self.intraatomic_context_net = nn.Sequential(
            snn.Dense(2 * n_atom_basis, n_atom_basis, activation=activation),
            snn.Dense(n_atom_basis, 3 * n_atom_basis, activation=None),
        )
        self.mu_channel_mix = snn.Dense(
            n_atom_basis, 2 * n_atom_basis, activation=None, bias=False
        )
        self.epsilon = epsilon

    def forward(self, q: torch.Tensor, mu: torch.Tensor):
        """Compute intraatomic mixing.

        Args:
            q: scalar input values
            mu: vector input values

        Returns:
            atom features after interaction
        """
        ## intra-atomic
        mu_mix = self.mu_channel_mix(mu)
        mu_V, mu_W = torch.split(mu_mix, self.n_atom_basis, dim=-1)
        mu_Vn = torch.sqrt(torch.sum(mu_V**2, dim=-2, keepdim=True) + self.epsilon)

        ctx = torch.cat([q, mu_Vn], dim=-1)
        x = self.intraatomic_context_net(ctx)

        dq_intra, dmu_intra, dqmu_intra = torch.split(x, self.n_atom_basis, dim=-1)
        dmu_intra = dmu_intra * mu_W

        dqmu_intra = dqmu_intra * torch.sum(mu_V * mu_W, dim=1, keepdim=True)

        q = q + dq_intra + dqmu_intra
        mu = mu + dmu_intra
        return q, mu


class PaiNN(nn.Module):
    """PaiNN - polarizable interaction neural network

    References:

    .. [#painn1] Schütt, Unke, Gastegger:
       Equivariant message passing for the prediction of tensorial properties and molecular spectra.
       ICML 2021, http://proceedings.mlr.press/v139/schutt21a.html

    """

    def __init__(
        self,
        n_atom_basis: int,
        n_interactions: int,
        radial_basis: nn.Module,
        cutoff_fn: Optional[Callable] = None,
        activation: Optional[Callable] = F.silu,
        max_z: int = 100,
        shared_interactions: bool = False,
        shared_filters: bool = False,
        epsilon: float = 1e-8,
    ):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
                This determines the size of each embedding vector; i.e. embeddings_dim.
            n_interactions: number of interaction blocks.
            radial_basis: layer for expanding interatomic distances in a basis set
            cutoff_fn: cutoff function
            activation: activation function
            shared_interactions: if True, share the weights across
                interaction blocks.
            shared_interactions: if True, share the weights across
                filter-generating networks.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNN, self).__init__()

        self.n_atom_basis = n_atom_basis
        self.n_interactions = n_interactions
        self.cutoff_fn = cutoff_fn
        self.cutoff = cutoff_fn.cutoff
        self.radial_basis = radial_basis

        self.embedding = nn.Embedding(max_z, n_atom_basis, padding_idx=0)

        self.share_filters = shared_filters

        if shared_filters:
            self.filter_net = snn.Dense(
                self.radial_basis.n_rbf, 3 * n_atom_basis, activation=None
            )
        else:
            self.filter_net = snn.Dense(
                self.radial_basis.n_rbf,
                self.n_interactions * n_atom_basis * 3,
                activation=None,
            )

        self.interactions = snn.replicate_module(
            lambda: PaiNNInteraction(
                n_atom_basis=self.n_atom_basis, activation=activation
            ),
            self.n_interactions,
            shared_interactions,
        )
        self.mixing = snn.replicate_module(
            lambda: PaiNNMixing(
                n_atom_basis=self.n_atom_basis, activation=activation, epsilon=epsilon
            ),
            self.n_interactions,
            shared_interactions,
        )

    def forward(self, inputs: Dict[str, torch.Tensor]):
        """
        Compute atomic representations/embeddings.

        Args:
            inputs (dict of torch.Tensor): SchNetPack dictionary of input tensors.

        Returns:
            torch.Tensor: atom-wise representation.
            list of torch.Tensor: intermediate atom-wise representations, if
            return_intermediate=True was used.
        """
        # get tensors from input dictionary
        atomic_numbers = inputs[properties.Z]
        r_ij = inputs[properties.Rij]
        idx_i = inputs[properties.idx_i]
        idx_j = inputs[properties.idx_j]
        n_atoms = atomic_numbers.shape[0]

        # compute atom and pair features
        d_ij = torch.norm(r_ij, dim=1, keepdim=True)
        dir_ij = r_ij / d_ij
        phi_ij = self.radial_basis(d_ij)
        fcut = self.cutoff_fn(d_ij)

        filters = self.filter_net(phi_ij) * fcut[..., None]
        if self.share_filters:
            filter_list = [filters] * self.n_interactions
        else:
            filter_list = torch.split(filters, 3 * self.n_atom_basis, dim=-1)

        q = self.embedding(atomic_numbers)[:, None]
        qs = q.shape
        mu = torch.zeros((qs[0], 3, qs[2]), device=q.device)

        for i, (interaction, mixing) in enumerate(zip(self.interactions, self.mixing)):
            q, mu = interaction(q, mu, filter_list[i], dir_ij, idx_i, idx_j, n_atoms)
            q, mu = mixing(q, mu)

        q = q.squeeze(1)

        inputs["scalar_representation"] = q
        inputs["vector_representation"] = mu
        return inputs


ModuleNotFoundError: No module named 'schnetpack'